In [2]:
import pandas as pd
twitter = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = 'latin1', header=None)

In [3]:
import re
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

def labelFilter(label):
  if label == 4:
    return 1
  return label

twitter=twitter[[0,5]]
twitter[5]=twitter[5].apply(preprocess)
twitter[0] = twitter[0].apply(labelFilter)
twitter

,0,5
0,0,"Awww, thats a bummer. You shoulda got Davi..."
1,0,is upset that he cant update his Facebook by t...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, its not behaving at all. im mad. why am i..."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com Very cool to hear old Walt intervi...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import tensorflow_text as text

In [5]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

sentences = ["I love you", "I hate you"]
preprocessed_text = bert_preprocess(sentences)
bert_encoder(preprocessed_text)["pooled_output"]

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-9.1058284e-01, -3.5070494e-01, -3.2318205e-01, ...,
        -4.3899752e-05, -7.1246153e-01,  9.1118795e-01],
       [-8.1203622e-01, -2.2017559e-01,  3.5842109e-01, ...,
         4.2030007e-01, -5.8162093e-01,  8.2115549e-01]], dtype=float32)>

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(twitter[5], twitter[0], test_size = .15)

example_x, example_y = x_train[:1000], y_train[:1000]

In [7]:
text_input = keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

dense = keras.layers.Dense(1, activation='sigmoid')(outputs["pooled_output"])

model = keras.Model(inputs=[text_input], outputs=[dense])
model.summary()
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(example_x, example_y, epochs=10)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  